In [1]:
import pandas as pd
import numpy as np
import re
import os
import glob
import warnings
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', None)

pd.set_option('display.float_format', '{:.0f}'.format)

warnings.filterwarnings('ignore')

### Import Data

In [3]:
df_india = pd.read_parquet('ecomm_india_may2024_v2.parquet', engine='pyarrow')
print('Completed')

Completed


In [4]:
df_india

,customer_id,customer_birthyear,age_group,customer_gender,id_order,order_date,year,month_year,order_region,loyalty_program,order_type,category_L2,category_L3,category_L4,category_L5,retailer,category,brand_group,brand_name,item_qty,item_value,projection_weight,total_order_qty,total_order_value,projected_total_order_qty,projected_total_order_value
0,6046158,1982,Gen. Y - [1980-1994],1,1113326465,2023-08-25,2023,2023_08,Top 8 metros,is amazon prime,standard,uncategorized,uncategorized,uncategorized,uncategorized,AMAZON,"BOOKS, SERVICES & OTHERS",brand not captured,brand not captured,1,197,758,2,394,1515,298395
1,4619273,1988,Gen. Y - [1980-1994],1,1197328524,2023-03-11,2023,2023_03,Top 8 metros,is amazon prime,standard,uncategorized,uncategorized,uncategorized,uncategorized,AMAZON,"BOOKS, SERVICES & OTHERS",brand not captured,brand not captured,1,222,263,2,445,527,117203
2,850915,1978,Gen. X - [1960-1979],NaN,1208510070,2023-09-04,2023,2023_09,Top 8 metros,is amazon prime,standard,uncategorized,uncategorized,uncategorized,uncategorized,AMAZON,"BOOKS, SERVICES & OTHERS",brand not captured,brand not captured,1,1884,1595,2,3768,3191,6011148
3,4830842,2000,Gen. Z - [1995-],1,1026729938,2023-08-14,2023,2023_08,Top 8 metros,is amazon prime,standard,uncategorized,uncategorized,uncategorized,uncategorized,AMAZON,"BOOKS, SERVICES & OTHERS",brand not captured,brand not captured,1,91,1114,4,364,4458,405650
4,4364187,1996,Gen. Z - [1995-],1,1208410958,2023-07-23,2023,2023_07,Rest of tier 1,is amazon prime,standard,uncategorized,uncategorized,uncategorized,uncategorized,AMAZON,"BOOKS, SERVICES & OTHERS",brand not captured,brand not captured,1,82,183,2,165,366,30204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8446876,4918454,1991,Gen. Y - [1980-1994],2,1379253456,2024-04-14,2024,2024_04,Top 8 metros,None,standard,baby products,nursing and feeding,breastfeeding,uncategorized,AMAZON,Electronics & Accessories,brand not captured,brand not captured,1,6469,282,1,6469,282,1823239
8446877,5839796,1997,Gen. Z - [1995-],1,1379153169,2024-04-14,2024,2024_04,Tier 2,is amazon prime,standard,beauty,skin care,moisturizing products,body,AMAZON,Consumables,beiersdorf,nivea,1,194,792,1,194,792,153993
8446878,4249257,1999,Gen. Z - [1995-],NaN,1379362532,2024-04-14,2024,2024_04,Rest of tier 1,is amazon prime,standard,computer and electronics,data storage,external solid state drives,uncategorized,AMAZON,Electronics & Accessories,first fishery development services,seagate,1,6395,901,1,6395,901,5764297
8446879,4680193,1981,Gen. Y - [1980-1994],1,1379686477,2024-04-14,2024,2024_04,Rest of tier 1,is amazon prime,standard,health and personal care,sex and sensuality,safer sex and contraception,condoms,AMAZON,Electronics & Accessories,brand not captured,brand not captured,1,190,552,1,190,552,104788


### Analysis

#### 0-a. Demo - Age

In [33]:
#age group별 소비력

df_grouped = df_india.groupby('age_group', dropna=False).agg({
    'projected_total_order_value': 'sum'
}).reset_index()

df_grouped

,age_group,projected_total_order_value
0,Gen. B - [-1959],108184396225
1,Gen. X - [1960-1979],1080142169086
2,Gen. Y - [1980-1994],3901905039208
3,Gen. Z - [1995-],4066968161501
4,unknown,262109992776


In [25]:
#age group별 projected buyer 수 산출

df_age_g = df_india[['customer_id','projection_weight','age_group']].drop_duplicates() #가중치가 부여된 패널 비중을 산출하기 위한 새로운 df 생성
df_age_g = df_age_g[df_age_g['projection_weight'].notnull()]  #projection_weight이 빈 패널은 유효하지 않은 패널로 간주되어 삭제함

df_age_g.groupby('age_group', dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

,age_group,projection_weight
0,Gen. B - [-1959],979987
1,Gen. X - [1960-1979],8418265
2,Gen. Y - [1980-1994],30386431
3,Gen. Z - [1995-],58555750
4,unknown,3488070


In [23]:
df_grouped = df_age_g.groupby('age_group', dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

fig = px.pie(
    df_grouped, 
    names='age_group',
    values='projection_weight',
    title='Projected Number of Online Buyers by Age Group',
    color='age_group',
    hole=0.3
)

fig.show()

#### 0-b. Demo - Region

In [36]:
#region group별 소비력

df_india.groupby('order_region', dropna=False).agg({
    'projected_total_order_value':'sum'
}).reset_index()

,order_region,projected_total_order_value
0,Rest of tier 1,1598611976705
1,Tier 2,2396132774624
2,Tier 3,1942844956439
3,Top 8 metros,3481720051027


In [37]:
#지역별 projected buyer 수 산출

df_region_g = df_india[['customer_id','projection_weight','order_region']].drop_duplicates() #가중치가 부여된 패널 비중을 산출하기 위한 새로운 df 생성
df_region_g = df_region_g[df_region_g['projection_weight'].notnull()]  #projection_weight이 빈 패널은 유효하지 않은 패널로 간주되어 삭제함

df_region_g.groupby('order_region', dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

,order_region,projection_weight
0,Rest of tier 1,34634175
1,Tier 2,46830622
2,Tier 3,32077179
3,Top 8 metros,46965910


In [39]:
df_grouped = df_region_g.groupby('order_region', dropna=False).agg({
    'projection_weight': 'sum'
}).reset_index()

fig = px.pie(
    df_grouped, 
    names='order_region',
    values='projection_weight',
    title='Projected Number of Online Buyers by Region',
    color='order_region',
    hole=0.3
)

fig.show()

#### 0-c. Demo - Gender

In [40]:
#gender별 소비력

df_india.groupby('customer_gender', dropna=False).agg({
    'projected_total_order_value':'sum'
}).reset_index()

,customer_gender,projected_total_order_value
0,1,5013582132982
1,2,2183537513648
2,NaN,2222190112167


In [41]:
#gender별 projected buyer 수 산출

df_gender_g = df_india[['customer_id','projection_weight','customer_gender']].drop_duplicates() #가중치가 부여된 패널 비중을 산출하기 위한 새로운 df 생성
df_gender_g = df_gender_g[df_gender_g['projection_weight'].notnull()]  #projection_weight이 빈 패널은 유효하지 않은 패널로 간주되어 삭제함

df_gender_g.groupby('customer_gender', dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

,customer_gender,projection_weight
0,1,58142774
1,2,19101153
2,NaN,24584575


In [32]:
df_india.groupby('customer_gender', dropna=False).agg({
    'projected_total_order_value':'sum'
}).reset_index()

,customer_gender,projected_total_order_value
0,1,5013582132982
1,2,2183537513648
2,NaN,2222190112167


#### 0-d. Sales - Category

##### Category Level 1

In [44]:
df_india.groupby('category', dropna=False).agg({
    'projected_total_order_value':'sum'
}).reset_index()

,category,projected_total_order_value
0,"BOOKS, SERVICES & OTHERS",1629622613025
1,Consumables,2469694304092
2,Electronics & Accessories,758912803406
3,Fashion,2286723815554
4,Home & Kitchen,455680928664
5,LA+TV,283275931128
6,Mobiles,1254816788487
7,NaN,280582574440


In [43]:
#category별 projected buyer 수 산출

df_cat_L1_g = df_india[['customer_id','projection_weight','category']].drop_duplicates() #가중치가 부여된 패널 비중을 산출하기 위한 새로운 df 생성
df_cat_L1_g = df_cat_L1_g[df_cat_L1_g['projection_weight'].notnull()]  #projection_weight이 빈 패널은 유효하지 않은 패널로 간주되어 삭제함

df_cat_L1_g.groupby('category', dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

,category,projection_weight
0,"BOOKS, SERVICES & OTHERS",73981237
1,Consumables,86693320
2,Electronics & Accessories,57255546
3,Fashion,78536281
4,Home & Kitchen,54562728
5,LA+TV,9265790
6,Mobiles,25009315
7,NaN,39867976


In [45]:
df_grouped = df_cat_L1_g.groupby('category', dropna=False).agg({
    'projection_weight': 'sum'
}).reset_index()

fig = px.pie(
    df_grouped, 
    names='category',
    values='projection_weight',
    title='Projected Number of Online Buyers by Category',
    color='category',
    hole=0.3
)

fig.show()

##### Category Level 2

In [49]:
df_india.category.unique()

array(['BOOKS, SERVICES & OTHERS', 'Fashion', 'Consumables',
       'Home & Kitchen', 'Electronics & Accessories', 'Mobiles', None,
       'LA+TV'], dtype=object)

In [50]:
df_india[df_india['category']=='Consumables'].category_L2.unique()

array(['grocery', 'beauty', 'cigarettes, tobacco and accessories',
       'health and personal care', 'baby products', 'home and kitchen',
       'pet supplies', 'uncategorized', 'sports and outdoors',
       'diy and tools', 'computer and electronics', 'automotive',
       'clothing', 'luggage and bags', 'jewellery', 'garden and outdoors',
       'lighting', 'toys and games', 'pc and video games',
       'musical instruments and dj', 'stationery and office supplies',
       'watches', 'large appliances', 'shoes and accessories', 'software',
       'books', 'business, industry and science', 'digital books',
       'dvd and blu-ray'], dtype=object)

In [52]:
# (df_india['category']=='Consumables') & (df_india['category_L2']=='beauty') 로 필터할 경우 우리가 필요한 데이터셋으로 활용 가능해보임
# category_L3까지 이용할 필요는 없어보임

df_india[
    (df_india['category']=='Consumables') &
    (df_india['category_L2']=='beauty')
].category_L3.unique()

array(['skin care', 'make-up', 'bath and body', 'hair care',
       'manicure and pedicure', 'fragrances',
       'personal grooming combination and travel packs',
       'multi-product boxes'], dtype=object)

In [55]:
df_beauty = df_india[
    (df_india['category']=='Consumables') &
    (df_india['category_L2']=='beauty')
].reset_index(drop=True)

df_beauty.head(3)

,customer_id,customer_birthyear,age_group,customer_gender,id_order,order_date,year,month_year,order_region,loyalty_program,order_type,category_L2,category_L3,category_L4,category_L5,retailer,category,brand_group,brand_name,item_qty,item_value,projection_weight,total_order_qty,total_order_value,projected_total_order_qty,projected_total_order_value
0,5584612,1996,Gen. Z - [1995-],1,1010276300,2023-08-03,2023,2023_08,Top 8 metros,None,standard,beauty,skin care,sun care and tanning,sun protections,REST,Consumables,honasa consumer,mamaearth,1,331,279,8,2097,2228,584194
1,6786813,1994,Gen. Y - [1980-1994],1,1314214098,2023-09-21,2023,2023_09,Top 8 metros,None,standard,beauty,skin care,moisturizing products,body,REST,Consumables,honasa consumer,mamaearth,1,352,279,8,1659,2228,462002
2,4955448,2001,Gen. Z - [1995-],NaN,1260455901,2023-12-03,2023,2023_12,Top 8 metros,None,standard,beauty,skin care,moisturizing products,body,REST,Consumables,unilever / unilever,vaseline (unilever / unilever),1,142,1027,8,1041,8218,1069360


In [57]:
df_beauty.groupby(['category','category_L2'], dropna=False).agg({
    'projected_total_order_value':'sum'
}).reset_index()

,category,category_L2,projected_total_order_value
0,Consumables,beauty,651503706380


In [58]:
#category별 projected buyer 수 산출

df_cat_beauty = df_beauty[['customer_id','projection_weight','category','category_L2']].drop_duplicates() #가중치가 부여된 패널 비중을 산출하기 위한 새로운 df 생성
df_cat_beauty = df_cat_beauty[df_cat_beauty['projection_weight'].notnull()]  #projection_weight이 빈 패널은 유효하지 않은 패널로 간주되어 삭제함

df_cat_beauty.groupby(['category','category_L2'], dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

,category,category_L2,projection_weight
0,Consumables,beauty,64076501


#### 0-e. Sales - Retailer

In [35]:
df_india.groupby(['retailer'], dropna=False).agg({
    'projected_total_order_value':'sum'
}).reset_index()

,retailer,projected_total_order_value
0,AJIO,541456897539
1,AMAZON,2703946164803
2,BIGBASKET,1218404611243
3,FLIPKART,2282050748900
4,MI.COM,25824890893
5,MYNTRA,1085516961765
6,NYKAA,342832228641
7,REST,1214180392271
8,SNAPDEAL,5096862741


In [60]:
#retailer별 projected buyer 수 산출

df_retailer_g = df_india[['customer_id','projection_weight','retailer']].drop_duplicates() #가중치가 부여된 패널 비중을 산출하기 위한 새로운 df 생성
df_retailer_g = df_retailer_g[df_retailer_g['projection_weight'].notnull()]  #projection_weight이 빈 패널은 유효하지 않은 패널로 간주되어 삭제함

df_retailer_g.groupby('retailer', dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

,retailer,projection_weight
0,AJIO,19723452
1,AMAZON,60958241
2,BIGBASKET,10278640
3,FLIPKART,37332226
4,MI.COM,1680602
5,MYNTRA,33010015
6,NYKAA,11271255
7,REST,53506155
8,SNAPDEAL,2306406


In [63]:
# Consumables-beauty 내 리테일러별 판매액

df_beauty.groupby(['category', 'category_L2', 'retailer']).agg({
    'projected_total_order_value':'sum'
}).reset_index()

,category,category_L2,retailer,projected_total_order_value
0,Consumables,beauty,AJIO,3359066978
1,Consumables,beauty,AMAZON,43957059992
2,Consumables,beauty,BIGBASKET,65274161228
3,Consumables,beauty,FLIPKART,56269266960
4,Consumables,beauty,MYNTRA,66561422119
5,Consumables,beauty,NYKAA,307739168308
6,Consumables,beauty,REST,108149785206
7,Consumables,beauty,SNAPDEAL,193775588


In [65]:
# Consumables-beauty 데이터 기준
#retailer별 projected buyer 수 산출

df_beauty_retailer_g = df_beauty[['customer_id','projection_weight','retailer']].drop_duplicates() #가중치가 부여된 패널 비중을 산출하기 위한 새로운 df 생성
df_beauty_retailer_g = df_beauty_retailer_g[df_beauty_retailer_g['projection_weight'].notnull()]  #projection_weight이 빈 패널은 유효하지 않은 패널로 간주되어 삭제함

df_beauty_retailer_g.groupby(['retailer'], dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

,retailer,projection_weight
0,AJIO,715203
1,AMAZON,26561377
2,BIGBASKET,4373871
3,FLIPKART,17053956
4,MYNTRA,9986034
5,NYKAA,10897658
6,REST,24310470
7,SNAPDEAL,367083


In [38]:
df_grouped = df_india.groupby(['retailer','category'], dropna=False).agg({
    'projected_total_order_value': 'sum'
}).reset_index()

fig = px.bar(df_grouped, 
             x='retailer', 
             y='projected_total_order_value', 
             color='category',
             title='Projected Total Order Value by Retailer X Category',
             labels={'projected_total_order_value': 'Projected Total Order Value'},
             height=500)

fig.show()


In [39]:
df_grouped = df_india.groupby(['retailer','category'], dropna=False).agg({
    'projected_total_order_value': 'sum'
}).reset_index()

for retailer in df_grouped['retailer'].unique():
    df_retailer = df_grouped[df_grouped['retailer'] == retailer]
    fig = px.bar(df_retailer, 
                 x='category', 
                 y='projected_total_order_value', 
                 title=f'Projected Total Order Value for {retailer}',
                 labels={'projected_total_order_value': 'Projected Total Order Value'})
    fig.show()


#### 0-f. Loyalty Program

In [71]:
df_india.groupby(['retailer','loyalty_program'], dropna=False).agg({'projected_total_order_value':'sum'}).reset_index()

,retailer,loyalty_program,projected_total_order_value
0,AJIO,NaN,541456897539
1,AMAZON,is amazon prime,1668388199475
2,AMAZON,NaN,1035557965328
3,BIGBASKET,NaN,1218404611243
4,FLIPKART,plus,1855721146272
5,FLIPKART,NaN,426329602629
6,MI.COM,NaN,25824890893
7,MYNTRA,myntra insider,604691953711
8,MYNTRA,NaN,480825008054
9,NYKAA,NaN,342832228641


In [70]:
#Loyalty program별 projected buyer 수 산출

df_LP_g = df_india[['customer_id','projection_weight','retailer','loyalty_program']].drop_duplicates() #가중치가 부여된 패널 비중을 산출하기 위한 새로운 df 생성
df_LP_g = df_LP_g[df_LP_g['projection_weight'].notnull()]  #projection_weight이 빈 패널은 유효하지 않은 패널로 간주되어 삭제함

df_LP_g.groupby(['retailer','loyalty_program'], dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

,retailer,loyalty_program,projection_weight
0,AJIO,NaN,19723452
1,AMAZON,is amazon prime,26302458
2,AMAZON,NaN,57980841
3,BIGBASKET,NaN,10278640
4,FLIPKART,plus,23523012
5,FLIPKART,NaN,24834219
6,MI.COM,NaN,1680602
7,MYNTRA,myntra insider,12327019
8,MYNTRA,NaN,28809435
9,NYKAA,NaN,11271255


In [72]:
df_beauty.groupby(['retailer','loyalty_program'], dropna=False).agg({'projected_total_order_value':'sum'}).reset_index()

,retailer,loyalty_program,projected_total_order_value
0,AJIO,NaN,3359066978
1,AMAZON,is amazon prime,27754340149
2,AMAZON,NaN,16202719843
3,BIGBASKET,NaN,65274161228
4,FLIPKART,plus,45875165843
5,FLIPKART,NaN,10394101117
6,MYNTRA,myntra insider,39510587636
7,MYNTRA,NaN,27050834484
8,NYKAA,NaN,307739168308
9,REST,NaN,108149785206


In [73]:
#Loyalty program별 projected buyer 수 산출

df_beauty_LP_g = df_beauty[['customer_id','projection_weight','retailer','loyalty_program']].drop_duplicates() #가중치가 부여된 패널 비중을 산출하기 위한 새로운 df 생성
df_beauty_LP_g = df_beauty_LP_g[df_beauty_LP_g['projection_weight'].notnull()]  #projection_weight이 빈 패널은 유효하지 않은 패널로 간주되어 삭제함

df_beauty_LP_g.groupby(['retailer','loyalty_program'], dropna=False).agg({
    'projection_weight':'sum'
}).reset_index()

,retailer,loyalty_program,projection_weight
0,AJIO,NaN,715203
1,AMAZON,is amazon prime,13911930
2,AMAZON,NaN,16244390
3,BIGBASKET,NaN,4373871
4,FLIPKART,plus,12395322
5,FLIPKART,NaN,6525154
6,MYNTRA,myntra insider,4223390
7,MYNTRA,NaN,6938912
8,NYKAA,NaN,10897658
9,REST,NaN,24310470


In [74]:
############################################################################################
'''
데이터프레임은,
total category의 경우 df_india
Consumables(L1) 내 beauty(L2) 가 필터된 데이터는 df_beauty 데이터프레임을 이용하시면 됩니다!
'''
############################################################################################

'\n데이터프레임은,\ntotal category의 경우 df_india\nConsumables(L1) 내 beauty(L2) 가 필터된 데이터는 df_beauty 데이터프레임을 이용하시면 됩니다!\n'

#### 1. 인도 화장품 시장

#### 2. 시장 세분화

##### 2-a. 시장 세분화 - 지역

##### 2-b. 시장 세분화 - age

##### 2-c. 시장 세분화 - others

#### 3. 주요 제품(positioning)

#### 4. Others

######################################################################################################

### (archive) Total India eComm Sales (Daily)

In [41]:
df_daily_sales = df_india.groupby(['order_date']).agg({'projected_total_order_value':'sum'}).reset_index()
df_daily_sales = df_daily_sales.sort_values(by='order_date').reset_index(drop=True)


fig = px.line(
    df_daily_sales, 
    x='order_date',
    y='projected_total_order_value',
    title='Projected Total Order Value Over Time'
)

fig.show()

### (archive) Progress Report

#### Descriptive Analysis

In [66]:
df_india.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8446881 entries, 0 to 8446880
Data columns (total 26 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   customer_id                  float64
 1   customer_birthyear           float64
 2   age_group                    object 
 3   customer_gender              float64
 4   id_order                     float64
 5   order_date                   object 
 6   year                         float64
 7   month_year                   object 
 8   order_region                 object 
 9   loyalty_program              object 
 10  order_type                   object 
 11  category_L2                  object 
 12  category_L3                  object 
 13  category_L4                  object 
 14  category_L5                  object 
 15  retailer                     object 
 16  category                     object 
 17  brand_group                  object 
 18  brand_name                   object 
 19  

In [67]:
df_india.describe()

,customer_id,customer_birthyear,customer_gender,id_order,year,item_qty,item_value,projection_weight,total_order_qty,total_order_value,projected_total_order_qty,projected_total_order_value
count,8446881,8194065,6510658,8446881,8446881,8429276,8446881,7962849,8446881,8443397,7962849,7960056
mean,4703459,1992,1,1173865476,2023,1,1007,616,5,1976,3766,1183322
std,1332603,8,0,192813636,0,6,3986,492,12,5146,11963,3855993
min,370,1898,1,637681478,2023,0,0,100,0,0,0,0
25%,4384386,1989,1,1012800468,2023,1,120,274,1,338,440,126645
50%,4794036,1993,1,1204732899,2023,1,299,395,2,749,1114,341334
75%,5320175,1997,2,1332760196,2024,1,650,1009,5,1721,2951,942678
max,7089483,2010,2,1461852426,2024,8000,396990,3991,8000,396990,3699938,886152048


In [68]:
age_groups = df_india['age_group'].unique()

for age_group in age_groups:
    data_age_group = df_india[df_india['age_group'] == age_group]
    
    category_totals = data_age_group.groupby('category')['projected_total_order_value'].sum().reset_index()
    
    fig = px.pie(category_totals, values='projected_total_order_value', names='category',
                 title=f"Total Spending by Category for Age Group: {age_group}",
                 hole=0.3)
    fig.show()